# Medical Appointment No Shows
This analysis is part of the Udacity Data Analysis Nanodegree program and aims to explore a dataset containing aproximately 100k medical appointments from the Brazilian public health system. 

## Table of Contents
<ul>
<li><a href="#intro">1. Introduction</a></li>
<li><a href="#wrangling">2. Data Wrangling</a></li>
<li><a href="#eda">3. Exploratory Data Analysis</a></li>
<li><a href="#conclusions">4. Conclusions</a></li>
<li><a href="#references">References</a></li>
</ul>

<a id='intro'></a>
## 1. Introduction

The Brazilian public health system, known as SUS for *Unified Health System* in its acronym in Portuguese, is one of the largest health system in the world <a href="#[1]">[1]</a>, representing government investment of more than 9% of GDP. However, its operation is not homogeneous and there are distinct perceptions of quality from citizens in different regions of the country.

This analysis will use the *no-show appointments dataset* which collects information from 100k medical appointments in Brazil and is focused on the question of whether or not patients show up for their appointment, with a number of characteristics about the patient included in each row:
- ‘ScheduledDay’: tells us on what day the patient set up their appointment.  
- ‘Neighborhood’: indicates the location of the hospital.  
- ‘Scholarship’: indicates whether or not the patient is enrolled in Brasilian welfare program called *Bolsa Família*.
- 'No-show': it says ‘No’ if the patient showed up to their appointment, and ‘Yes’ if they did not show up.

This analysis aims to outline the reasons for patient no-showing to scheduled appointments, as well as get insights about the Brazilian public health system. To accomplish this, we will first try to understand the data and the context on which they were collected. We will then proceed to data wrangling and exploratory data analysis, in an iterative process, in order to draw some conclusions about the subject.

### 1.1. Initial statements
This section sets up import statements for all the packages that will be used throughout this notebook.

In [1]:
# Data analysis packages:
import pandas as pd
import numpy as np
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### 1.2. Understanding the data
Since the original dataset is available at Kaggle <a href="#[2]">[2]</a>, the first step was to get additional information available on its original repository, as transcripted below:

>In response here it goes: Country? Brazil Is this from private or public service? Or both? Public Sector (Brazil) Primary, secondary, or tertiary care? Primary care, not sure about it. Only physicians? What would be the specialties? We don't have these information.
  
>First appointment with that specific professional? --> I believe first appointments are prone to "no show". That is a very good hint, we don't have that information yet. Part of the day? --> Early morning might be worse. (Perhaps in the next version we can include it.

Another relevant point is that any information is provided about the source of data. I.e., did these data come from a government agency or were they a compilation from each health unit? Furthermore, besides it is known the data is about public health unit, it is not said from which government sphere they belong to (municipal, state or federal). 
The absence of such information can result in a biased analysis, since the complex and decentralized Brazilian health system brings inevitable conflicts and contradictions between its associated entities, as identified in <a href="#[3]">[3]</a>.

This question regarding the data source was made at the Kaggle discussion forum (available at https://goo.gl/JB4P4d), on which the following answer were given: 
> This is at municipal level combining the appointments occurred in more then 45 units. As far as we know, the city has 20 million loss a year with no-shows. After so many studies happening in this data the average no-show now is 20%. Its important to mention that all services offered to the municipality were free of charge.

In order to define the hypotheses that will lead this analysis, it is first needed to know which information is available in the dataset (attributes). Thus, the first lines of the dataset are shown below:

In [2]:
pd.read_csv('noshowappointments-kagglev2-may-2016.csv').head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


### 1.3. Hypotheses to be explored

From the dataset information and its attributes, the following questions can be formulated:
* **Regarding not showing to appointment**:
  - Do the patients forget the appointment?
  - What is the average waiting time between the scheduling date and the appointment date?
  - What is the *missing appointments per patient* relation in the dataset?  
  - How many patients have missed an appointment at least once?
  
* **Regarding the patient profile**:
  - Is there any common characteristics among those patients that miss appointments?
  - Patients suffering from serious illnesses are more assiduous?

* **Regarding the health unit**:
  - What are the neighbourhoods with the highest no-showing rate?
  - Considering there are common characteristics among those who do not attend the appointments, how these characteristics are geographically distributed?
  
* **Regarding the data**:
  - Is the data balanced in relation to the interest class (showing/no-showing to appointments)?
  - How is the data distributed in relation to the location (neighborhood)? I.e. all the units have an equivalent and balanced number of instances?
  
The analysis conduced in the next sections will try to address these questions and to answer them based on the available dataset.

<a id='wrangling'></a>
## 2. Data Wrangling
In this section the data will be loaded and some operations will be performed to inspect data types, to look for missing values or possibly errant data. Data cleaning operations will be executed in the same section where a specific attribute is being analyzed.

### 2.1. General Properties

In [3]:
## Loading the dataset and printing out a few lines:
dataset = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')
dataset.head(3)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No


In [4]:
## Reading dataset general information:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


From the command above, it is possible to notice the dataset has no missing values, since all the attributes have 110.527 instances. However there is no guarantee these values are all consistent. In the way to check it, the *pandas.describe()* method will be applied in the next section to gather information from the numerical attributes.

### 2.2. Numerical attributes and outliers

In [5]:
## Describing the numerical attributes:
dataset.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


#### 2.2.1 PatientID:
This attribute type is originally numerical *(float64 type)*. However, since it represents the patient identification, any numerical operations should not be applied to it and therefore we will transform it into a string type:

In [6]:
## Checking the attribute data type:
type(dataset['PatientId'][0])

numpy.float64

In [7]:
## Converting the values to int type and then to str type:
dataset['PatientId'] = dataset['PatientId'].apply(lambda x: str(int(x)));

In [8]:
## Counting how many unique patients are in the dataset:
len(dataset['PatientId'].unique())

62299

Counting the unique values of this dataset attribute shows us how many unique patients we are dealing with. In this case, there are 62.298 patients which indicates that some patients had more than one appointment *(an average of 1.77 appointments per patient)*. 

#### 2.2.2 AppointmentID:
The same can be said for the *AppointmentID* attribute, which will also be transformed into a string type:

In [9]:
## Checking the attribute data type:
type(dataset['AppointmentID'][0])

numpy.int64

In [10]:
## Converting the values to int type and then to str type:
dataset['AppointmentID'] = dataset['AppointmentID'].apply(lambda x: str(int(x)));

In [11]:
## Counting how many unique patients are in the dataset:
len(dataset['AppointmentID'].unique())

110527

This attribute seems to be consistent: there are 110.526 instances in the dataset, as well as 110.526 unique values for the appointmentIDs. For last, since each instance corresponds to an appointment register, we will redefine the dataset index to the appointment IDs. 

In [12]:
dataset.set_index('AppointmentID', drop=True, inplace=True)

#### 2.2.3. Checking the *Age* attribute:
It is not possible to be aged $-1$, as well as older than $100$ years calls some attention. The latter will be analyzed appropriately at the exploratory data analysis section, but agest under zero are definitely a mistake. In this case, we will first check how many instances correspond to this before treat them.

In [13]:
dataset[dataset['Age']<0]

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
AppointmentID,,,,,,,,,,,,,
5775010,465943158731293,F,2016-06-06T08:58:13Z,2016-06-06T00:00:00Z,-1,ROMÃO,0,0,0,0,0,0,No


Since there is just one occurrence, it will be removed from the dataset.

In [14]:
dataset.drop('5775010',inplace=True)  #Removing the anomalous instance
# dataset.reset_index(drop=True,inplace=True)  #Reseting the dataset index

#### 2.2.4. Checking the *Handcap* attribute:
In the dataset metadata it is said that the *handcap* attribute should cointain a boolean value. However, as seen above, this attribute assumes values from 0 to 4, probably indicating the handicap number for each patient. In this analysis, we will map to 1 any value higher than 0.

In [15]:
## Converting all 'Handcap' values higher than 0 to 1:
dataset['Handcap'] = np.where(dataset['Handcap']>0, 1, 0)

#### Verifying the numerical attributes:
After the numerical data cleaning, we will check if is there any still visible inconsistencyand then proceed to treat the categorical variables.

In [16]:
dataset.describe()

,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,110526.000000,110526.000000,110526.000000,110526.000000,110526.000000,110526.000000,110526.000000
mean,37.089219,0.098266,0.197248,0.071865,0.030400,0.020276,0.321029
std,23.110026,0.297676,0.397923,0.258266,0.171686,0.140943,0.466874
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,115.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


As it can be seen, the numerical attributes are consistent with the expected values (i.e. counting 110.526 values, with a minimum of 0 and maximum value of 1 to the boolean attributes, while in a range from 0 to 115 for the age attribute).

### 2.3. Categorical attributes

In [17]:
## Getting information of the categorical attributes:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110526 entries, 5642903 to 5629448
Data columns (total 13 columns):
PatientId         110526 non-null object
Gender            110526 non-null object
ScheduledDay      110526 non-null object
AppointmentDay    110526 non-null object
Age               110526 non-null int64
Neighbourhood     110526 non-null object
Scholarship       110526 non-null int64
Hipertension      110526 non-null int64
Diabetes          110526 non-null int64
Alcoholism        110526 non-null int64
Handcap           110526 non-null int64
SMS_received      110526 non-null int64
No-show           110526 non-null object
dtypes: int64(7), object(6)
memory usage: 11.8+ MB


####  2.3.1. *Gender* attribute:
Only two values are expected from this attribute. In order to check its consistency, the sum for both **M**ale and **F**emale classes must equal the total number of instances.

In [18]:
## Counting gender classes
dataset.Gender.value_counts()

F    71839
M    38687
Name: Gender, dtype: int64

Although the attribute data are consistent (the total sums 110.526 instances), they are unbalanced. Since each instance represents the marking and not the patient, these data should be explored further and considered in the analysis.

####  2.3.2. Schedule and appointment date:
These two attributes are related to when the patient scheduled *(ScheduledDay)* and to which date he scheduled the appointment *(AppointmentDay)*. It is effort to notice, however, that the *AppointmentDay* has just the date information, while the *ScheduledDay* has both date and time informations.  
In this section we will parse the information in order to find out how many days have passed between the schedule date and the appointment date. To get to it, we could use the python native datetime library. However, we choose to use the pandas methods for datetime processing.

In [19]:
## Reading again the dataset first lines to get acquainted with its content:
dataset.head(2)

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
AppointmentID,,,,,,,,,,,,,
5642903,29872499824296,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
5642503,558997776694438,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No


##### Parsing the date and time information:

In [26]:
## Converting the date information in string to datetime type:
dataset['ScheduledDay'] = pd.to_datetime(dataset.ScheduledDay)
dataset['AppointmentDay'] = pd.to_datetime(dataset.AppointmentDay)
## Creating a new column (attribute) containing just the scheduling time:
dataset['ScheduleTime'] = dataset.ScheduledDay.dt.time
## Normalizing the "Day" columns to keep just the date information (dropping the time info)
dataset['ScheduledDay'] = dataset.ScheduledDay.dt.normalize()

A relevant information for this analysis is the waiting time, in days, between the scheduling date and the appointment date. In this way, a new column will be created to contain this data:

In [29]:
dataset['WaitingDays'] = dataset['AppointmentDay'] - dataset['ScheduledDay']

##### Checking data consistency

Then, the dataset consistency in relation to the scheduling and appointment times can be checked by the new *WaitingDays* attribute: If there are negative attributes, this indicates that the scheduling was performed after the appointment and such record should therefore be handled.  

To help further analysis, a function will be created to return a *time delta* type from a the input number of days. 

In [61]:
def waiting_days(days):
    '''Auxiliary function to parse a date information from string type to python datetime object.
    Syntax: waiting_days(days), where:
        days = int type with the number of days considered.
    Return: a correspondent pandas._libs.tslib.Timedelta data type.
    '''
    arg = str(days) + ' days'
    return pd.tslib.Timedelta(arg)

In [62]:
## Checking which instances were scheduled after the appointment:
dataset[dataset['WaitingDays'] < waiting_days(0)]

,PatientId,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduleTime,WaitingDays
AppointmentID,,,,,,,,,,,,,,,
5679978,7839272661752,M,2016-05-10,2016-05-09,38,RESISTÊNCIA,0,0,0,0,1,0,Yes,10:51:53,-1 days
5715660,7896293967868,F,2016-05-18,2016-05-17,19,SANTO ANTÔNIO,0,0,0,0,1,0,Yes,14:50:41,-1 days
5664962,24252258389979,F,2016-05-05,2016-05-04,22,CONSOLAÇÃO,0,0,0,0,0,0,Yes,13:43:58,-1 days
5686628,998231581612122,F,2016-05-11,2016-05-05,81,SANTO ANTÔNIO,0,0,0,0,0,0,Yes,13:49:20,-6 days
5655637,3787481966821,M,2016-05-04,2016-05-03,7,TABUAZEIRO,0,0,0,0,0,0,Yes,06:50:57,-1 days


Since few inconsistent instances have been found, they will be removed:

In [73]:
## Recording the inconsistent instances index 
dropIx = dataset[dataset['WaitingDays'] < waiting_days(0)].index
## Dropping these instances from the dataset:
dataset.drop(dropIx, inplace=True)

##### Rearranging the dataset attributes
Just in order to help its reading, the dataset attributes will be rearranged to put all the date and time information together.

In [77]:
## Reading the dataset attributes (columns):
dataset.columns

Index(['PatientId', 'Gender', 'ScheduledDay', 'AppointmentDay', 'Age',
       'Neighbourhood', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handcap', 'SMS_received', 'No-show', 'ScheduleTime',
       'WaitingDays'],
      dtype='object')

In [81]:
dataset.reindex(columns=['PatientId', 'Gender', 'Age', 'Scholarship', 'Hipertension', 'Diabetes',
       'Alcoholism', 'Handcap', 'ScheduledDay', 'ScheduleTime', 'AppointmentDay', 'WaitingDays', 'SMS_received', 
       'Neighbourhood', 'No-show'])

,PatientId,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,ScheduledDay,ScheduleTime,AppointmentDay,WaitingDays,SMS_received,Neighbourhood,No-show
AppointmentID,,,,,,,,,,,,,,,
5642903,29872499824296,F,62,0,1,0,0,0,2016-04-29,18:38:08,2016-04-29,0 days,0,JARDIM DA PENHA,No
5642503,558997776694438,M,56,0,0,0,0,0,2016-04-29,16:08:27,2016-04-29,0 days,0,JARDIM DA PENHA,No
5642549,4262962299951,F,62,0,0,0,0,0,2016-04-29,16:19:04,2016-04-29,0 days,0,MATA DA PRAIA,No
5642828,867951213174,F,8,0,0,0,0,0,2016-04-29,17:29:31,2016-04-29,0 days,0,PONTAL DE CAMBURI,No
5642494,8841186448183,F,56,0,1,1,0,0,2016-04-29,16:07:23,2016-04-29,0 days,0,JARDIM DA PENHA,No
5626772,95985133231274,F,76,0,1,0,0,0,2016-04-27,08:36:51,2016-04-29,2 days,0,REPÚBLICA,No
5630279,733688164476661,F,23,0,0,0,0,0,2016-04-27,15:05:12,2016-04-29,2 days,0,GOIABEIRAS,Yes
5630575,3449833394123,F,39,0,0,0,0,0,2016-04-27,15:39:58,2016-04-29,2 days,0,GOIABEIRAS,Yes
5638447,56394729949972,F,21,0,0,0,0,0,2016-04-29,08:02:16,2016-04-29,0 days,0,ANDORINHAS,No


#### 2.3.3. *Neighborhood* names

####  2.3.4. *No-show* class:
We want to check wheter the class values are consistent with the expected values for this database (true or false). However, in order to facilitate its manipulation by pandas libray, we will first rename this attribute to *No_show*. 

In [ ]:
## Renaming the dataset "No-show" attribute to "No_show":
dataset.columns = ['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No_show']

In [ ]:
## Counting gender classes
dataset.No_show.value_counts()

The total sum is consistent with the dataset length. However, as it was noticed for the *Gender* attribute, this data are unbalanced, i.e. there are more instances pertaining to one class than other. Since this attribute represents our dataset categorization, this fact becomes more relevant for the further analysis. In other words, although we can get some useful insights from this data, we must take care when applying this dataset to training machine learning algorithms and so on. 

#### Data wrangling overview
We conclude this analysis step by showing an overview of the pre-processed data, i.e. the data after the wrangling and cleansing process. 

In [ ]:
## Checking again the dataset information (for numerical attributes) and description (for categorical ones):
print(dataset.info())
dataset.describe()

<a id='eda'></a>
## 3. Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!

<a id='references'></a>
## References

<a id='[1]'></a>[1] Wikipedia contributors. Brazil. Wikipedia, The Free Encyclopedia. February 9, 2018, 22:06 UTC. Available at: https://en.wikipedia.org/w/index.php?title=Brazil&oldid=824851232. Accessed February 10, 2018.

<a id='[2]'></a>[2] Kaggle. *Medical Appointment No Shows*. Reviewed dataset. Available at: https://www.kaggle.com/joniarroba/noshowappointments. Accessed February 10, 2018.

<a id='[3]'></a>[3] Victora, C. G., Barreto, M. L., Leal, M. D. C., Monteiro, C. A., Schmidt, M. I., Paim, J. S., ... & Reichenheim, M. E. (2011). *Condições de saúde e inovações nas políticas de saúde no Brasil: o caminho a percorrer* (in Portuguese). Available at: https://repositorio.ufba.br/ri/handle/ri/5973. Accessed February 12, 2018.

### Backlog (Verificar a utilidade disso na análise)

> Aqui o ponto principal será identificar de onde são esses dados. Das informações disponíveis no Kagle, já sabe-se que são de Vitória, ES. Portanto, vai ser muito interessante cruzar com os dados demográficos dessa cidade, a partir do IBGE.
>

##### About the Introduction Section
> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

### Data Cleaning (Replace this with more specific notes!)

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.


In [ ]:
dataset['Neighbourhood'].unique()

#### Scheduling Time and appointment...

In [ ]:
def parse_date(date):
    '''Auxiliar function to parse a date information from string type to python datetime object.
    Syntax: parse_date(date), where:
        date = string with the date information.'''
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

##### Ages over hundred

In [ ]:
dataset.describe()

In [ ]:
## Plotting the attribute distribution:
plt.hist(dataset['Age'], bins=115);

In [ ]:
sns.distplot(dataset['Age'])